In [3]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, SparsePauliOp
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, RuntimeOptions
import numpy as np
import matplotlib.pyplot as plt

# Initialize the runtime service
service = QiskitRuntimeService()

# Prepare the initial Bell state |Ψ⟩ = (|01⟩ + |10⟩)/√2
circuit = QuantumCircuit(2)
circuit.x(1)  # Prepare |10⟩
circuit.h(0)  # Apply Hadamard to the first qubit
circuit.cx(0, 1)  # Apply CNOT to create entanglement

# Hamiltonian Parameters
omega0 = 1.0  # Energy level splitting
kappa = 0.1   # Coupling strength between qubits

# Define Pauli operators as SparsePauliOp for tensor operations
Z = SparsePauliOp.from_list([('Z', 1)])
X = SparsePauliOp.from_list([('X', 1)])
Y = SparsePauliOp.from_list([('Y', 1)])
I = SparsePauliOp.from_list([('I', 1)])

# Construct the Hamiltonian using tensor products
H0 = omega0 / 2 * (Z.tensor(I) + I.tensor(Z))
HI = kappa * ((X.tensor(X)) - (Y.tensor(Y)))
H = H0 + HI

# Define time points for simulation
time_points = np.linspace(0, 10, 100)

# Initialize the Sampler
backend = service.backend("ibmq_qasm_simulator")
sampler = Sampler(backend)

# Define options for running the Sampler
options = {'backend': 'ibmq_qasm_simulator'}

# Run the simulation
results = []
for t in time_points:
    job = service.run(
        program=sampler,
        inputs={
            'circuits': circuit,
            'observable': H,
            'parameter_values': [[t]]
        },
        options=options
    )
    result = job.result()
    # Calculate concurrence for the evolved state
    concurrence_value = concurrence(Statevector(result))  # Ensure you define concurrence computation
    results.append(concurrence_value)

# Plotting the concurrence over time
plt.figure(figsize=(10, 6))
plt.plot(time_points, results, label='Concurrence')
plt.xlabel('Time')
plt.ylabel('Concurrence')
plt.title('Time Evolution of Concurrence')
plt.legend()
plt.grid(True)
plt.show()


/var/folders/s8/49z4mhw56f1gk77hx70jgjsw0000gn/T/ipykernel_28442/3732285502.py:46: DeprecationWarning: Circuits that do not match the target hardware definition will no longer be supported after March 1, 2024. See the transpilation documentation (https://docs.quantum.ibm.com/transpile) for instructions to transform circuits and the primitive examples (https://docs.quantum.ibm.com/run/primitives-examples) to see this coupled with operator transformations.
  job = sampler.run(


RuntimeJobFailureError: "Unable to retrieve job result. Unexpected program input parameter 'observable'."